In [168]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import html5lib
import lxml
import os
from os.path import join
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from time import sleep
import re

In [154]:
browser = webdriver.Chrome('/home/sandeep/Desktop/Projects/Python/Patna/chromedriver')

In [155]:
url = 'http://patnahighcourt.gov.in/CaseAdvSearch.aspx'

### Access by case_type

1. Extract all case type options from dropdownlist
2. Make an iterator of the case type options to aid the manual aspect
3. Fill in the captcha manually
4. Extract response from table -- including the url fragment
5. Write to dataframe



#### Step 1

In [156]:
browser.get(url)

In [157]:
case_type_select = Select(browser.find_element_by_name("ctl00$MainContent$ddlCaseType"))
case_types = iter([x.text for x in case_type_select.options])

#### Step 2

In [164]:
case_type_select = Select(browser.find_element_by_name("ctl00$MainContent$ddlCaseType"))
case_type = next(case_types)
case_type_select.select_by_visible_text(case_type)

#### Step 3

#### Step 4

In [165]:
# Helper function to make dataframe

def append_to_table(headers, rows, new_column):
    new_table = []
    for row in rows:
        new_table.append([x.text.strip() for x in row.find_all('td')])
    df = pd.DataFrame(new_table, columns=headers)
    df['url'] = new_column
    return df

In [167]:
table_soup = BeautifulSoup(browser.page_source).find('table', attrs={'id':"ctl00_MainContent_gvSearch"})
header = [x.text.strip() for x in table_soup.find_all('tr')[0].find_all('th')]
rows = table_soup.find_all('tr')[1:]
link_elements = browser.find_elements_by_link_text('VIEW')
delay = 5

l_rows = len(rows)
assert l_rows==len(link_elements)
urls = []
for i in range(l_rows):
    # refresh link_elements
    print('%d out of %d' %((i+1), l_rows))
    link_elements = browser.find_elements_by_link_text('VIEW')
    link_elements[i].click()
    sleep(2)
    try:
        myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'ctl00_MainContent_fvCaseStatusCri_HyperLink1')))
        url_text = BeautifulSoup(browser.page_source).find('a', attrs={'id':"ctl00_MainContent_fvCaseStatusCri_HyperLink1"})['href'].replace('CaseStatus.aspx?CNO=', '')
        urls.append(url_text)

    except TimeoutException:
        urls.append(None)

    

1 out of 8184
2 out of 8184
3 out of 8184
4 out of 8184
5 out of 8184


KeyboardInterrupt: 

In [163]:
df = append_to_table(header, rows, urls)


df.to_csv(join('data','cases_accessed_by_case_type', case_type.replace('.', '')+'.csv'), index=False)